### Helper Classes

First we get all of our helper modules. The prepare_EMG module will prepare the EMG data for phoneme recognition. The prepare_outputs module will prepare our target labels and align them with our EMG data. The module 'prepare_data' will help us read data from CSV into a dataframe. Finally, 'vis' will help visualize EMG data in both time and frequency domains. 

In [3]:
%load_ext autoreload
%autoreload 2

import prepare_EMG, prepare_outputs, prepare_data, vis
# autodetector = Output_Prep.detector
EMG_Prep = prepare_EMG.EMG_preparer(window_size=30.0)
# Output_Prep = prepare_outputs.output_preparer(subvocal_detector = autodetector, window_size=30.0)
Output_Prep = prepare_outputs.output_preparer(window_size=30.0)

Data_Prep = prepare_data.data_preparer()



/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Score: 0.696146245059


### Labeling the Data

First, we need to visualize a few EMG voltage graphs to find some sections that most likely contain no subvocalization. Then, we'll need to find some regions that almost certainly do. These two classes of EMG readouts will serve to train an identifier to help us automatically label EMG windows with phonemes. The model used here will most likely be an SVC, inside "prepare_outputs". It will process each EMG window in order, and when it finds one that most likely contains subvocalization, it applies the next phoneme as that window's label. 

In [4]:
data_1 = Data_Prep.load('Sat Mar  4 00:44:23 2017')
data_2 = Data_Prep.load('Sat Mar  4 00:44:23 2017')
data_3 = Data_Prep.load('Sat Mar  4 00:44:23 2017')
data_4 = Data_Prep.load('Sat Mar  4 00:44:23 2017')
data_5 = Data_Prep.load('Sat Mar  4 00:44:23 2017')
data_6 = Data_Prep.load('Sat Mar  4 00:44:23 2017')


In [11]:
import pandas
%autoreload 2



labels_frame = pandas.read_csv('austen_subvocal.csv')
labels_frame
# print(labels_frame.iloc[0][0])
trans_labels_1 = Output_Prep.transform(labels_frame.iloc[0][0])
data_1_proc = EMG_Prep.process(data_1)
# print("This is trans_labels:",trans_labels,'This is trans_labels length:',len(trans_labels))
# print('this is data_1_proc:',data_1_proc)
aligned_data_1, trans_labels_1 = Output_Prep.zip(data_1_proc, trans_labels_1, repeat=3)
# print('this is aligned_data', aligned_data)
aligned_data, trans_labels = aligned_data_1, trans_labels_1
# aligned_data = aligned_data_1.append([aligned_data_2, aligned_data_3])
# trans_labels = trans_labels_1.append([trans_labels_2, trans_labels_3])

we make labels longer by one whole
we make labels longer by one whole
              manner      place     height vowel
DH  voiced-fricative     dental        max    no
AH             vowel        mid        mid   yes
F          fricative     labial        max    no
AE             vowel  mid-front        low   yes
M              nasal     labial        max    no
AH             vowel        mid        mid   yes
L        approximant    lateral  very high    no
IY             vowel      front  very high   yes
AH             vowel        mid        mid   yes
V   voiced-fricative     labial        max    no
D        voiced-stop   alveolar        max    no
AE             vowel  mid-front        low   yes
SH         fricative      front        max    no
W        approximant       back  very-high    no
UH             vowel   mid-back       high   yes
D        voiced-stop   alveolar        max    no
HH         aspirated     uknown        max    no
AE             vowel  mid-front        low   yes

### AF Extractor Models

These models will be optimized for extracting AF's from the data, before passing those AF's onto an MLPC for identifying the most likely phoneme. 

In [12]:
# Prepare lists of parameters for our GridSearch
# First, our layer sizes
layer_sizes = []
for i in range(1,7):
    for j in range(0,310,10):
        if j:
            tup = []
            for k in range(i):
                tup.append(j)
            layer_sizes.append(tuple(tup))
print(layer_sizes)

# Next, our alpha values
alphas = [0.0000001,0.0000006,0.000001,0.000006,0.00001,0.00006,0.0001,0.0006,0.001,0.006,0.01,0.06,0.1,0.6,1,6,10,16,100,160,1000,1600]




[(10,), (20,), (30,), (40,), (50,), (60,), (70,), (80,), (90,), (100,), (110,), (120,), (130,), (140,), (150,), (160,), (170,), (180,), (190,), (200,), (210,), (220,), (230,), (240,), (250,), (260,), (270,), (280,), (290,), (300,), (10, 10), (20, 20), (30, 30), (40, 40), (50, 50), (60, 60), (70, 70), (80, 80), (90, 90), (100, 100), (110, 110), (120, 120), (130, 130), (140, 140), (150, 150), (160, 160), (170, 170), (180, 180), (190, 190), (200, 200), (210, 210), (220, 220), (230, 230), (240, 240), (250, 250), (260, 260), (270, 270), (280, 280), (290, 290), (300, 300), (10, 10, 10), (20, 20, 20), (30, 30, 30), (40, 40, 40), (50, 50, 50), (60, 60, 60), (70, 70, 70), (80, 80, 80), (90, 90, 90), (100, 100, 100), (110, 110, 110), (120, 120, 120), (130, 130, 130), (140, 140, 140), (150, 150, 150), (160, 160, 160), (170, 170, 170), (180, 180, 180), (190, 190, 190), (200, 200, 200), (210, 210, 210), (220, 220, 220), (230, 230, 230), (240, 240, 240), (250, 250, 250), (260, 260, 260), (270, 270, 

In [13]:
from sklearn.neural_network import MLPClassifier as MLPC
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV
import copy

param_grid = {
    'solver':['lbfgs','sgd','adam'],
    'hidden_layer_sizes':layer_sizes,
    'activation':['identity','logisitic','tanh','relu'],
    'alpha': alphas,
    'learning_rate':['constant','invscaling','adaptive'],
    'max_iter':[100,200,300,400,500,600,700,800],
    'power_t':[(1.0/n) for n in range(1,1000,10)],
    'momentum':[(1.0/n) for n in range(1,1000,10)],
    'beta_1':[((10000-n)/10000) for n in range(2,1000,10)],
    'beta_2':[((10000-n)/10000) for n in range(2,100,5)]
}
grid_search = GridSearchCV(MLPC(), param_grid, n_jobs=-1)
# vectorizer = DictVectorizer()
# trans_aligned_labels = vectorizer.fit_transform([item[1] for item in aligned_labels.iterrows()])
# print(trans_aligned_labels)
# TODO: Implement feature vectorization to use multioutput classification of MLPC
manner_classifier = MLPC(solver='lbfgs',hidden_layer_sizes=(21,21,30),random_state=3)
manner_classifier.fit(aligned_data, trans_labels['manner'])
m_score = manner_classifier.score(aligned_data, trans_labels['manner'])

place_classifier = MLPC(solver='lbfgs',hidden_layer_sizes=(30,30,30),random_state=6)
place_classifier.fit(aligned_data, trans_labels['place'])
p_score = place_classifier.score(aligned_data, trans_labels['place'])

height_classifier = MLPC(solver='lbfgs',hidden_layer_sizes=(30,30,30),random_state=9)
height_classifier.fit(aligned_data, trans_labels['height'])
h_score = height_classifier.score(aligned_data, trans_labels['height'])

vowel_classifier = MLPC(solver='lbfgs',hidden_layer_sizes=(30,30,30),random_state=12)
vowel_classifier.fit(aligned_data, trans_labels['vowel'])
v_score = vowel_classifier.score(aligned_data, trans_labels['vowel'])

print('manner score:',m_score,'place score:',p_score,'height score:',h_score,'vowel score:',v_score)
print(data_1_proc.head(50), trans_labels['manner'].head(50))

manner score: 0.371794871795 place score: 0.320512820513 height score: 0.50641025641 vowel score: 0.666666666667
     0.000000  0.033333  0.033333  0.066667  0.066667      0.100000  \
0   45.336328  3.246606  1.194433  2.065605  1.649199  1.478745e+00   
1   49.190625  0.194345  0.002563  0.099662  0.002144  6.749619e-02   
2   48.984375  0.218216  0.026103  0.030316  0.180943  7.966844e-03   
3   49.255078  0.145012  0.099569  0.033292  0.079299  5.062214e-02   
4   48.842578  0.124892  0.009580  0.240587  0.085307  2.976467e-02   
5   48.958594  0.005331  0.123206  0.210249  0.193549  8.437024e-02   
6   48.907031  0.037701  0.001253  0.273924  0.045263  1.131945e-01   
7   48.326953  0.138992  0.111664  0.051443  0.155449  2.791139e-01   
8   48.584766  0.028369  0.086189  0.030830  0.019087  1.136909e-02   
9   48.816797  0.002073  0.058700  0.040098  0.011378  7.966844e-03   
10  48.507422  0.005252  0.204882  0.226363  0.048907  4.193893e-01   
11  48.455859  0.005331  0.115226  

In [ ]:
manner_classifier2 = copy.deepcopy(grid_search)
manner_classifier2.fit(aligned_data, trans_labels['manner'])
m_score2 = manner_classifier2.score(aligned_data, trans_labels['manner'])

print('manner score:',m_score2)

/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/py

In [82]:
place_classifier2 = MLPC(solver='lbfgs',hidden_layer_sizes=(240,240),random_state=6)
place_classifier2.fit(aligned_data, trans_labels['place'])
p_score2 = place_classifier2.score(aligned_data, trans_labels['place'])

print('place score:',p_score2)

place score: 0.301282051282


In [83]:
height_classifier2 = MLPC(solver='lbfgs',hidden_layer_sizes=(60,60),random_state=9)
height_classifier2.fit(aligned_data, trans_labels['height'])
h_score2 = height_classifier2.score(aligned_data, trans_labels['height'])

print('height score:',h_score2)

height score: 0.532051282051


In [84]:
vowel_classifier2 = MLPC(solver='lbfgs',hidden_layer_sizes=(240,240,240),random_state=12)
vowel_classifier2.fit(aligned_data, trans_labels['vowel'])
v_score2 = vowel_classifier2.score(aligned_data, trans_labels['vowel'])

print('vowel score:',v_score2)

vowel score: 0.647435897436
